## Validação do dataset

In [7]:
import pandas as pd
import os

# --- Configuração ---
dataset_path = r'C:\Github\Workspace GL\Retail Sales Forecasting Data'

# Função auxiliar para carregar um CSV com tratamento de erros
def load_csv_to_df(filename, base_path):
    file_path = os.path.join(base_path, filename)
    try:
        # Tenta com a codificação padrão UTF-8
        return pd.read_csv(file_path, encoding='utf-8', on_bad_lines='skip')
    except UnicodeDecodeError:
        print(f"Aviso: Usando codificação 'latin1' para o arquivo {filename}")
        # Tenta com latin1 se UTF-8 falhar
        return pd.read_csv(file_path, encoding='latin1', on_bad_lines='skip')
    except FileNotFoundError:
        print(f"ERRO: Arquivo não encontrado: {file_path}")
        return None
    except Exception as e:
        print(f"ERRO ao ler {filename}: {e}")
        return None

# --- Carregamento dos Dados em variáveis separadas ---
print("Carregando cada CSV em uma variável de DataFrame dedicada...")

actual_matrix_df = load_csv_to_df('actual_matrix.csv', dataset_path)
catalog_df = load_csv_to_df('catalog.csv', dataset_path)
discounts_history_df = load_csv_to_df('discounts_history.csv', dataset_path)
markdowns_df = load_csv_to_df('markdowns.csv', dataset_path)
online_df = load_csv_to_df('online.csv', dataset_path)
price_history_df = load_csv_to_df('price_history.csv', dataset_path)
sales_df = load_csv_to_df('sales.csv', dataset_path)
stores_df = load_csv_to_df('stores.csv', dataset_path)

print("\n✅ DataFrames carregados!")


✅ DataFrames carregados!


In [ ]:
# --- Print do cabeçalho de todos os DataFrames carregados ---
dataframes = [
    ("actual_matrix_df", actual_matrix_df),
    ("catalog_df", catalog_df),
    ("discounts_history_df", discounts_history_df),
    ("markdowns_df", markdowns_df),
    ("online_df", online_df),
    ("price_history_df", price_history_df),
    ("sales_df", sales_df),
    ("stores_df", stores_df)
]

for name, df in dataframes:
    if df is not None:
        print(f"\n--- Cabeçalho do DataFrame '{name}' ---")
        print(df.head())
    else:
        print(f"\n--- DataFrame '{name}' não foi carregado. ---")

In [8]:
display(catalog_df)

,Unnamed: 0,item_id,dept_name,class_name,subclass_name,item_type,weight_volume,weight_netto,fatness
0,0,da17e2d5feda,БУМАЖНО-ВАТНАЯ ПРОДУКЦИЯ,БУМАЖНАЯ ПРОДУКЦИЯ,ВЛАЖНЫЕ САЛФЕТКИ,NaN,150.0,NaN,NaN
1,1,614de2b96018,БУМАЖНО-ВАТНАЯ ПРОДУКЦИЯ,ВАТНАЯ ПРОДУКЦИЯ,ВАТНЫЕ ДИСКИ,NaN,30.0,NaN,NaN
2,2,0c1f1f3e3e11,БУМАЖНО-ВАТНАЯ ПРОДУКЦИЯ,ВАТНАЯ ПРОДУКЦИЯ,ВАТНЫЕ ДИСКИ,NaN,NaN,NaN,NaN
3,3,71a7fa99f005,ТОВАРЫ ДЛЯ ДОМА,ПОСУДА,КУХОННАЯ ПОСУДА,NaN,NaN,NaN,NaN
4,4,ec1bd4d59fe9,БУМАЖНО-ВАТНАЯ ПРОДУКЦИЯ,ВАТНАЯ ПРОДУКЦИЯ,ВАТНЫЕ ДИСКИ,NaN,30.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
219805,219806,a8eae2f5974a,СЛАДКИЕ МУЧНЫЕ ИЗДЕЛИЯ,ПЕЧЕНЬЕ,ПЕЧЕНЬЕ ФАСОВАННОЕ,NaN,NaN,NaN,NaN
219806,219807,54a786e4a589,СЛАДКИЕ МУЧНЫЕ ИЗДЕЛИЯ,ПЕЧЕНЬЕ,ПЕЧЕНЬЕ ФАСОВАННОЕ,NaN,NaN,NaN,NaN
219807,219808,a96e686cb2ff,ДЕТСКИЙ МИР,ИГРУШКИ,РАЗВИВАЮЩИЕ ИГРУШКИ,NaN,NaN,NaN,NaN
219808,219809,49cdb76c6eb7,ДЕТСКИЙ МИР,ИГРУШКИ,РАЗВИВАЮЩИЕ ИГРУШКИ,NaN,NaN,NaN,NaN


In [9]:
from langdetect import detect
import pandas as pd

# Função para detectar idioma
def detect_language(text):
    try:
        return detect(str(text))
    except:
        return "unknown"  # Retorna "unknown" se a detecção falhar

# Detectar idioma de cada coluna
for col in ['dept_name', 'class_name', 'subclass_name']:
    if col in catalog_df.columns:
        catalog_df[f'{col}_lang'] = catalog_df[col].apply(detect_language)

# Exibir idiomas detectados
print(catalog_df[['dept_name', 'dept_name_lang', 'class_name', 'class_name_lang', 'subclass_name', 'subclass_name_lang']])

: 

: 

In [ ]:
display(catalog_df)

## Validando tradução

### Transformers (Hugging Face)

In [ ]:
from transformers import pipeline

# Pipeline multilingue para traduzir para português
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-pt")

# Traduzindo as colunas desejadas
for col in ['dept_name', 'class_name', 'subclass_name']:
    if col in catalog_df.columns:
        catalog_df[f'{col}_pt'] = catalog_df[col].astype(str).apply(lambda x: translator(x)[0]['translation_text'])
    else:
        print(f"Coluna '{col}' não encontrada em catalog_df.")

# Exibindo as colunas originais e traduzidas
display(catalog_df[['dept_name', 'dept_name_pt', 'class_name', 'class_name_pt', 'subclass_name', 'subclass_name_pt']])

In [ ]:
from langdetect import detect
import pandas as pd

# Função para detectar idioma
def detect_language(text):
    try:
        return detect(str(text))
    except:
        return "unknown"  # Retorna "unknown" se a detecção falhar

# Exemplo de DataFrame
catalog_df = pd.DataFrame({
    'dept_name': ['Electronics', 'Ropa', 'Livres'],
    'class_name': ['Gadgets', 'Ropa casual', 'Fiction'],
    'subclass_name': ['Smartphones', 'Camisetas', 'Romans']
})

# Detectar idioma de cada coluna
for col in ['dept_name', 'class_name', 'subclass_name']:
    if col in catalog_df.columns:
        catalog_df[f'{col}_lang'] = catalog_df[col].apply(detect_language)

# Exibir idiomas detectados
print(catalog_df[['dept_name', 'dept_name_lang', 'class_name', 'class_name_lang', 'subclass_name', 'subclass_name_lang']])

In [ ]:
display(catalog_df)